In [34]:
#bibliotecas

import requests
import zipfile
import io
import pandas as pd
import numpy as np
from difflib import get_close_matches
import unidecode


In [33]:
# variaveis

start_y = 2011
final_y = 2022
codecvm = '19348'
#arquivo com data da publicação de relatórios
event_dates_file =  'c:\\Users\\thgcn\\OneDrive\\Academico\\Financial-Reports-Impact\\data\\itr_date_itau.csv'
#arquivo com preços históricos
itau_file = 'c:\\Users\\thgcn\\OneDrive\\Academico\\Financial-Reports-Impact\\data\\\historical_data\\ITUB4.SA.csv'


In [36]:
# funções

def busca_date_pub(ano, codecvm):
    """
    Busca dados de publicação no site da CVM para o ano e código CVM especificados.

    Args:
        ano (int): O ano dos dados.
        codecvm (str): O código CVM da empresa.

    Returns:
        list: Lista de tuplas contendo os campos Assunto e Data_Entrega para o código CVM especificado.

    Example:
        >>> busca_date_pub(2023, '019348')
        [('Informações consolidadas dos exercícios', '2023-05-15'), ...]
    """
    url = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/IPE/DADOS/ipe_cia_aberta_%d.zip' % ano
    r = requests.get(url)
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    file = zf.namelist()
    zf = zf.open(file[0])
    lines = zf.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in lines]
    lines = [i.split(';') for i in lines]

    # Header mapping to find indexes of relevant columns
    header = lines[0]
    idx_codigo_cvm = header.index('Codigo_CVM')
    idx_assunto = header.index('Assunto')
    idx_data_entrega = header.index('Data_Entrega')

    # Strings de busca normalizadas para evitar problemas com acentos e maiúsculas/minúsculas
    busca1 = unidecode.unidecode("Informações consolidadas dos exercícios").lower()
    busca2 = unidecode.unidecode("Informações sobre o resultado").lower()

    result = []
    for line in lines[1:]:
        if line[idx_codigo_cvm] == codecvm:
            assunto_normalizado = unidecode.unidecode(line[idx_assunto]).lower()
            if busca1 in assunto_normalizado or busca2 in assunto_normalizado:
                result.append((line[idx_codigo_cvm], line[idx_assunto], line[idx_data_entrega]))

    # Convertendo o resultado em um DataFrame
    df_result = pd.DataFrame(result, columns=['Codigo_CVM', 'Assunto', 'Data_Entrega'])

    return df_result

def save_cvm_dates_combined(start_y, final_y, codecvm, event_dates_file):
    combined_data = pd.DataFrame()  # DataFrame vazio para acumular os resultados
    
    # Loop para chamar a função para os anos de start_y a final_y
    for year in range(start_y, final_y + 1):
        # Obter os dados para o ano especificado
        df = busca_date_pub(year, codecvm)
        
        # Adicionar os dados ao DataFrame acumulado
        combined_data = pd.concat([combined_data, df], ignore_index=True)
    
    # Salvar os dados acumulados como um único arquivo CSV
    combined_data.to_csv(event_dates_file, index=False)
    print(f'Dados combinados (Codigo_CVM, Assunto e Data_Entrega) salvos em {event_dates_file}')    

def process_itau_data(itau_file, event_dates_file, codecvm):
    # Carregar dados do arquivo ITUB4.SA.csv
    itau_data = pd.read_csv(itau_file)
    itau_data['Date'] = pd.to_datetime(itau_data['Date'])  # Converter a coluna Date para o tipo datetime

    # Calcular o retorno diário em escala logarítmica
    itau_data['Return'] = np.log(itau_data['Close'] / itau_data['Close'].shift(1))

    # Carregar datas de eventos do arquivo itr_date_itau.csv
    event_dates = pd.read_csv(event_dates_file)
    event_dates['Data_Entrega'] = pd.to_datetime(event_dates['Data_Entrega'])  # Converter a coluna Data_Entrega para o tipo datetime

    # Definir as datas de eventos como um conjunto para busca eficiente
    event_set = set(event_dates['Data_Entrega'])

    # Determinar se cada data é um evento (1) ou não (0)
    itau_data['event'] = itau_data['Date'].isin(event_set).astype(int)

    # Receber o codcvm via parâmetro de entrada da função
    itau_data['CD_CVM'] = codecvm

    # Selecionar as colunas relevantes para o DataFrame final
    final_data = itau_data[['CD_CVM', 'Date', 'Return', 'event']]
    
    return final_data

def analyze_data_quality(final_data):
    # Identificar quantos "COD_CVM" distintos existem na base e exibir os valores
    unique_cod_cvm = final_data['CD_CVM'].unique()
    num_unique_cod_cvm = len(unique_cod_cvm)
    print(f"Existem {num_unique_cod_cvm} COD_CVM distintos na base:")
    print(unique_cod_cvm)
    
    # Identificar quantas datas iguais existem em "Date" e exibir os valores
    date_counts = final_data['Date'].value_counts()
    duplicate_dates = date_counts[date_counts > 1]
    print(f"\nExistem {len(duplicate_dates)} datas repetidas na base:")
    print(duplicate_dates)
    
    # Contar quantos eventos iguais a 0 e 1 existem na base
    event_0_count = (final_data['event'] == 0).sum()
    event_1_count = (final_data['event'] == 1).sum()
    print(f"\nContagem de eventos:")
    print(f"Eventos iguais a 0: {event_0_count}")
    print(f"Eventos iguais a 1: {event_1_count}")
    
    # Identificar quais colunas contêm valores NaN e quantas vezes
    nan_counts = final_data.isna().sum()
    columns_with_nan = nan_counts[nan_counts > 0]
    print(f"\nColunas com valores NaN e a quantidade de NaNs:")
    print(columns_with_nan)


In [37]:
#chamada da função
save_cvm_dates_combined(start_y, final_y, codecvm, event_dates_file)
final_data = process_itau_data(itau_file, event_dates_file, codecvm)
analyze_data_quality(final_data)

Dados combinados (Codigo_CVM, Assunto e Data_Entrega) salvos em c:\Users\thgcn\OneDrive\Academico\Financial-Reports-Impact\data\itr_date_itau.csv
Existem 1 COD_CVM distintos na base:
['19348']

Existem 0 datas repetidas na base:
Series([], Name: Date, dtype: int64)

Contagem de eventos:
Eventos iguais a 0: 3180
Eventos iguais a 1: 45

Colunas com valores NaN e a quantidade de NaNs:
Return    1
dtype: int64


In [39]:
combined_data = pd.DataFrame()
for year in range(start_y, final_y + 1):
    # Obter os dados para o ano especificado
    df = busca_date_pub(year, codecvm)
        
    # Adicionar os dados ao DataFrame acumulado
    combined_data = pd.concat([combined_data, df], ignore_index=True)

In [40]:
combined_data

,Codigo_CVM,Assunto,Data_Entrega
0,19348,Informações consolidadas dos exercícios findos...,2011-02-22
1,19348,Informações consolidadas dos exercícios findos...,2011-05-03
2,19348,Informações consolidadas dos exercícios findos...,2011-08-02
3,19348,Informações consolidadas dos exercícios findos...,2011-11-01
4,19348,Informações consolidadas dos exercícios findos...,2012-02-07
5,19348,Informaçoes Consolidadas dos exercícios findos...,2012-04-24
6,19348,Informações consolidadas dos exercícios findos...,2012-07-24
7,19348,Informações consolidadas dos exercícios findos...,2012-10-23
8,19348,Informações consolidadas dos exercícios findos...,2013-02-05
9,19348,Informações consolidadas dos exercícios findos...,2013-04-30
